# <center>Online Retail Clustering</center>

## Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go

## Reading dataframe

In [ ]:
# Pulling invoice data
df_retail = pd.read_csv(r'C:\Users\Prashant\Documents\Machine Learing Workspace\Online-Retail-Clustering\.gitignore\OnlineRetail.csv')

In [ ]:
df_retail.head()

## Null value treatment

In [ ]:
df_retail.info()

In [ ]:
print('Number of na in customer id =' ,df_retail['CustomerID'].isna().sum())
print('Number of null in customer id =' ,df_retail['CustomerID'].isnull().sum())

In [ ]:
df_retail['CustomerID'].fillna(0,inplace=True)
df_retail['Description'].fillna('No description',inplace=True)
df_retail.info()

## Mathematical implementation of K means clustering techinique
1.Pick K points as the initial centroids from the data set, either randomly or the first K.<br>
2.Find the Euclidean distance of each point in the data set with the identified K points — cluster centroids.<br>
3.Assign each data point to the closest centroid using the distance found in the previous step.<br>
4.Find the new centroid by taking the average of the points in each cluster group.<br>
5.Repeat iteration till the centroids don’t change.<br>


In [ ]:
# Defining centroids
k = 5

# Filtering the columns required for clustering
df_retail_clust = df_retail.loc[:,['Quantity','UnitPrice']]

#Copying dataframe
data=df_retail_clust
data.head()

In [ ]:
# Storing the sample dataframe to determine the number of centroids
k_means = (data.sample(k, replace=False))    # store current means
k_means2 = pd.DataFrame()                    # store previous means
clusters = pd.DataFrame()    

In [ ]:
while not k_means2.equals(k_means):
    # distance matrix (euclidean distance)
    cluster_count = 0
    for idx, k_mean in k_means.iterrows():
        clusters[cluster_count] = (data[k_means.columns] - np.array(k_mean)).pow(2).sum(1).pow(0.5)
        cluster_count += 1

    # update cluster
    data['MDCluster'] = clusters.idxmin(axis=1)

    # store previous cluster
    k_means2 = k_means
    k_means = pd.DataFrame()
    k_means_frame = data.groupby('MDCluster').agg(np.mean)
    k_means[k_means_frame.columns] = k_means_frame[k_means_frame.columns]

## Plotting clusters 

In [ ]:
# plotting
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,10))
plt.scatter(data.iloc[:,0],data.iloc[:,1],c=data.iloc[:,2] ,s=50, cmap='viridis')
plt.show()